In [8]:
# 라이브러리 로드
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, LassoCV
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, classification_report, precision_recall_curve
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE

# Google Drive에서 데이터 로드
from google.colab import drive
drive.mount('/content/gdrive')

train_data = pd.read_csv("/content/gdrive/My Drive/ml/Training_diabetes_binary_health_indicators_BRFSS2015.csv")
test_data = pd.read_csv("/content/gdrive/My Drive/ml/Test_diabetes_binary_5050split_health_indicators_BRFSS2015.csv")

# 변수 선택
necessary_features = ['Fruits', 'Veggies']
reference_features = ['BMI', 'Smoker', 'Age', 'PhysActivity', 'Sex', 'Education', 'Income']
target = 'Diabetes_binary'
selected_features = necessary_features + reference_features

train_data = train_data[[target] + selected_features]
test_data = test_data[[target] + selected_features]

# 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(
    train_data[selected_features],
    train_data[target],
    test_size=0.3,
    random_state=42
)

# SMOTE 비율 조정 (소수 클래스 비율을 0.75로 설정)
smote = SMOTE(sampling_strategy=0.75, random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

# 특성 스케일링
scaler = StandardScaler()
X_train[selected_features] = scaler.fit_transform(X_train[selected_features])
X_val[selected_features] = scaler.transform(X_val[selected_features])

# 라쏘 교차검증으로 중요한 변수 선택
lasso_model = LassoCV(cv=5, random_state=42)
lasso_model.fit(X_train, y_train)
lasso_selected_features = np.array(selected_features)[lasso_model.coef_ != 0]


print(f"Number of selected features by Lasso: {len(lasso_selected_features)}")


# 라쏘로 선택된 변수만 사용
X_train_lasso = X_train[lasso_selected_features]
X_val_lasso = X_val[lasso_selected_features]

# Logistic Regression 하이퍼파라미터 튜닝
param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
grid = GridSearchCV(LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced'),
                    param_grid, cv=5)
grid.fit(X_train_lasso, y_train)

print(f"\nBest C: {grid.best_params_['C']}")

# 최적의 하이퍼파라미터로 모델 학습
best_model = grid.best_estimator_
best_model.fit(X_train_lasso, y_train)

# 검증 데이터에 대한 예측
y_val_pred_proba = best_model.predict_proba(X_val_lasso)[:, 1]

# Threshold 최적화
thresholds = np.linspace(0.1, 0.9, 9)
best_f1 = 0
best_threshold = 0.5

for threshold in thresholds:
    y_val_pred_custom = (y_val_pred_proba >= threshold).astype(int)
    f1 = f1_score(y_val, y_val_pred_custom)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

print(f"\nBest Threshold: {best_threshold}")
print(f"Best F1 Score: {best_f1:.3f}")

# 최적화된 Threshold 적용
y_val_pred = (y_val_pred_proba >= best_threshold).astype(int)

# 최종 성능 평가
auc_score = roc_auc_score(y_val, y_val_pred_proba)
accuracy = accuracy_score(y_val, y_val_pred)
f1 = f1_score(y_val, y_val_pred)

print("\nFinal Model Performance:")
print(f"AUC: {auc_score:.3f}")
print(f"Accuracy: {accuracy:.3f}")
print(f"F1 Score: {f1:.3f}")


# 부트스트래핑
bootstrap_iterations = 100
bootstrap_scores = []

for _ in range(bootstrap_iterations):
    bootstrap_sample_X, bootstrap_sample_y = resample(X_val_lasso, y_val, random_state=42)
    y_pred_proba = best_model.predict_proba(bootstrap_sample_X)[:, 1]
    auc = roc_auc_score(bootstrap_sample_y, y_pred_proba)
    bootstrap_scores.append(auc)

mean_auc = np.mean(bootstrap_scores)
std_auc = np.std(bootstrap_scores)

print("\nBootstrapping Results:")
print(f"Mean AUC: {mean_auc:.3f}")
print(f"AUC Standard Deviation: {std_auc:.3f}")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


Number of selected features by Lasso: 9

Best C: 10

Best Threshold: 0.6
Best F1 Score: 0.389

Final Model Performance:
AUC: 0.765
Accuracy: 0.769
F1 Score: 0.389

Bootstrapping Results:
Mean AUC: 0.766
AUC Standard Deviation: 0.000


1. SMOTE 비율 조정: 소수 클래스(1)의 샘플을 다수 클래스(0)의 75%까지만 증강하여 클래스 간 균형을 맞춤
2. 라쏘로 선택된 변수만 사용
3. Threshold 최적화(기본 임계값(0.5) 최적화하여 F1 스코어가 최대가 되는 값을 찾음)
4. 하이퍼파라미터 튜닝